In [2]:
import pandas as pd
from ggplot import *
import numpy as np
%matplotlib inline

# Set ipython's max row display
pd.set_option('display.max_row', 1000)

# Set iPython's max column width to 50
pd.set_option('display.max_columns', 50)

In [3]:
df = pd.read_csv('world_war.csv')

# Quick glance at the data

In [4]:
df.head(10)

,Name,Year,Assaulter,Defender,Assaulter_1,Defender_1,Assaulter_strength,Defender_strength,Outcome
0,Baltic Operation,1941,Nazi Germany,Soviet Union,NaN,NaN,787 500,498 000,win
1,Battle of Bialystok–Minsk,1941,Nazi Germany,Soviet Union,NaN,NaN,750 000,675 000,win
2,Battle of Brody,1941,Nazi Germany,Soviet Union,NaN,NaN,750,3500,win
3,Battle of Hanko,1941,Nazi Germany,Soviet Union,Finland,NaN,25 300,30 000,loss
4,Demyansk Pocket,1942,Soviet Union,Nazi Germany,NaN,NaN,130 000,400 000,loss


In [5]:
#Create list of Assaulters
assaulter_list = []

for elem in df['Assaulter'].unique():
    if type(elem) == str:
        assaulter_list.append(elem)
    else:
        continue

#Create list of Defenders
defender_list = []

for elem in df['Defender'].unique():
    if type(elem) == str:
        defender_list.append(elem)
    else:
        continue
        
print(defender_list)

['Soviet Union', 'Nazi Germany']


In [10]:
#How many times has been successfully carried out the attack
wins = []

for elem in assaulter_list:
    score = 0
    for row in df['Assaulter'][df['Outcome'] == 'win']:
        if elem in row:
            score +=1
        else:
            continue
    wins.append(score)

data = {'Assaulter': assaulter_list,
        'Win': wins}
attack_win = pd.DataFrame(data)

In [11]:
#How many times has been unsuccessfully carried out the attack
loss = []

for elem in assaulter_list:
    score = 0
    for row in df['Assaulter'][df['Outcome'] == 'loss']:
        if elem in row:
            score +=1
        else:
            continue
    loss.append(score)

data = {'Assaulter': assaulter_list,
        'Loss': loss}
attack_loss = pd.DataFrame(data)

In [14]:
#How many times has been successfully carried out protection
wins = []

for elem in defender_list:
    score = 0
    for row in df['Defender'][df['Outcome'] == 'loss']:
        if elem in row:
            score +=1
        else:
            continue
    wins.append(score)

data = {'Defender': defender_list,
        'Win': wins}
protection_win = pd.DataFrame(data)

In [15]:
#How many times has been unsuccessfully carried out protection
loss = []

for elem in defender_list:
    score = 0
    for row in df['Defender'][df['Outcome'] == 'win']:
        if elem in row:
            score +=1
        else:
            continue
    loss.append(score)

data = {'Defender': defender_list,
        'Loss': loss}
protection_loss = pd.DataFrame(data)